In [1]:
import pickle
import numpy as np
import pandas as pd
import re

seed = 2023

In [2]:
import torch

# set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
from nlpsig_networks.scripts.swmhau_network_functions import (
    swmhau_network_hyperparameter_search
)

In [4]:
output_dir = "talklife_moc_output"
if not os.path.isdir(output_dir):
    os.makedirs(output_dir)

Talklife MoC

In [5]:
%run load_talklifemoc.py

In [6]:
%run load_sbert-embeddings.py

In [7]:
sbert_embeddings.shape

torch.Size([18604, 384])

swmhau Network

In [8]:
features = ["time_encoding", "timeline_index"]
standardise_method = ["z_score", None]
include_features_in_path = True
include_features_in_input = False

In [9]:
#create indices for kfold
fold_col_names = [c for c in df.columns if 'fold' in c ]
fold_list = []
for foldc in fold_col_names:
    fold_list.append((df[df[foldc]=='train'].index, df[df[foldc]=='dev'].index, df[df[foldc]=='test'].index))
fold_list = tuple(fold_list)

In [10]:
num_epochs = 100
embedding_dim = 384
dimensions = [15] 
dimreduction_method = ["umap"]
# define swmhau parameters: (output_channels, sig_depth, num_heads)
swmhau_parameters = [(12, 3, 10), (8, 4, 6)]
num_layers = [1]
ffn_hidden_dim_sizes = [[256,256],[512,512]]
dropout_rates = [0.1, 0.2] 
learning_rates = [1e-3, 1e-4, 5e-4]
seeds = [1, 12, 123]
loss = "focal"
gamma = 2
validation_metric = "f1"
patience = 5

In [11]:
size = 20
swmhau_network_umap_20, best_swmhau_network_umap_20, _, __ = swmhau_network_hyperparameter_search(
    num_epochs=num_epochs,
    df=df,
    id_column="timeline_id",
    label_column="label",
    embeddings=sbert_embeddings.numpy(),
    y_data=y_data,
    output_dim=output_dim,
    history_lengths=[size],
    dim_reduce_methods=dimreduction_method,
    dimensions=dimensions,
    log_signature=True,
    swmhau_parameters=swmhau_parameters,
    num_layers=num_layers,
    ffn_hidden_dim_sizes=ffn_hidden_dim_sizes,
    dropout_rates=dropout_rates,
    learning_rates=learning_rates,
    seeds=seeds,
    loss=loss,
    gamma=gamma,
    device=device,
    features=features,
    standardise_method=standardise_method,
    include_features_in_path=include_features_in_path,
    include_features_in_input=include_features_in_input,
    split_ids=None, 
    split_indices=fold_list,
    k_fold=True,
    patience=patience,
    validation_metric=validation_metric,
    results_output=f"{output_dir}/swmhau_network_umap_focal_{gamma}_{size}.csv",
    verbose=False
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


##################################################
dimension: 15 | method: umap
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/18604 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/18604 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.
saving results dataframe to CSV for this hyperparameter search in talklife_moc_output/swmhau_network_umap_focal_2_20.csv
saving the best model results dataframe to CSV for this hyperparameter search in talklife_moc_output/swmhau_network_umap_focal_2_20_best_model.csv


In [12]:
best_swmhau_network_umap_20[['f1', 'f1_scores', 'precision', 
       'recall', 'valid_f1',
       'valid_f1_scores', 'valid_precision', 
       'valid_recall', 'sig_depth',
       'num_heads', 
       'ffn_hidden_dim', 'dropout_rate', 'learning_rate', 'seed',
       'loss_function', 'k_fold', 'augmentation_type',
       'hidden_dim_aug', 'comb_method', 'batch_size']]

,f1,f1_scores,precision,recall,valid_f1,valid_f1_scores,valid_precision,valid_recall,sig_depth,num_heads,ffn_hidden_dim,dropout_rate,learning_rate,seed,loss_function,k_fold,augmentation_type,hidden_dim_aug,comb_method,batch_size
0,0.546047,"[0.8885513943193202, 0.4525178201885491, 0.297...",0.533342,0.561624,0.546671,"[0.8858965630279648, 0.47508247959715233, 0.27...",0.532496,0.564784,3,10,"(512, 512)",0.1,0.0005,1,focal,True,Conv1d,None,concatenation,64
0,0.544796,"[0.8888030142015264, 0.4383880736081994, 0.307...",0.534926,0.556351,0.543392,"[0.8860288572721332, 0.4652462451973455, 0.278...",0.532127,0.557513,3,10,"(512, 512)",0.1,0.0005,12,focal,True,Conv1d,None,concatenation,64
0,0.533093,"[0.8855252541552364, 0.43962432915921285, 0.27...",0.524947,0.544231,0.535730,"[0.883419233186675, 0.44354293441514536, 0.280...",0.528848,0.547358,3,10,"(512, 512)",0.1,0.0005,123,focal,True,Conv1d,None,concatenation,64


In [13]:
best_swmhau_network_umap_20["f1"].mean()

0.5413118496975763

In [14]:
best_swmhau_network_umap_20["precision"].mean()

0.5310719302727974

In [15]:
best_swmhau_network_umap_20["recall"].mean()

0.5540683802435522

In [16]:
np.stack(best_swmhau_network_umap_20["f1_scores"]).mean(axis=0)

array([0.88762655, 0.44351007, 0.29279892])

In [17]:
np.stack(best_swmhau_network_umap_20["precision_scores"]).mean(axis=0)

array([0.89961691, 0.41176063, 0.28183825])

In [18]:
np.stack(best_swmhau_network_umap_20["recall_scores"]).mean(axis=0)

array([0.87595517, 0.48082011, 0.30542986])